<a href="https://colab.research.google.com/github/Noctivar/montecarlo-whiteboard-pricer/blob/main/Monte_Carlo_Whiteboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
#  required packages
!pip install -q ipycanvas ipywidgets matplotlib numpy scipy

# Enable widget extensions for Colab
from google.colab import output
output.enable_custom_widget_manager()

# Test if ipycanvas is working
try:
    from ipycanvas import Canvas
    print("✅ ipycanvas installed successfully")
except ImportError:
    print("❌ ipycanvas installation failed")

✅ ipycanvas installed successfully


In [36]:
import numpy as np
import matplotlib.pyplot as plt
from ipycanvas import Canvas, hold_canvas
from ipywidgets import VBox, HBox, Button, Output
from scipy.interpolate import interp1d
import warnings
warnings.filterwarnings('ignore')

# Monte Carlo parameters
S0 = 1.0      # Initial stock price
MU = 0.0      # Drift (risk-neutral)
SIGMA = 0.20  # Volatility
N_PATHS = 10000
T = 1.0       # Time to expiration

# Canvas settings
CANVAS_WIDTH = 600
CANVAS_HEIGHT = 300

print("✅ All imports successful")

✅ All imports successful


In [41]:
# Global variables
drawn_points = []
is_drawing = False

# Create canvas
canvas = Canvas(width=CANVAS_WIDTH, height=CANVAS_HEIGHT)

def setup_canvas():
    """Draw the coordinate system"""
    with hold_canvas(canvas):
        # Clear and set white background
        canvas.clear()
        canvas.fill_style = 'white'
        canvas.fill_rect(0, 0, CANVAS_WIDTH, CANVAS_HEIGHT)

        # Draw grid lines
        canvas.stroke_style = 'lightgray'
        canvas.line_width = 1

        # Vertical line at x=300 (S=1.0)
        canvas.stroke_line(300, 0, 300, CANVAS_HEIGHT)
        # Horizontal line at y=150 (payoff=0)
        canvas.stroke_line(0, 150, CANVAS_WIDTH, 150)

        # Add labels
        canvas.fill_style = 'black'
        canvas.font = '12px Arial'
        canvas.fill_text('S=0.5', 10, 145)
        canvas.fill_text('S=1.0', 290, 145)
        canvas.fill_text('S=1.5', 440, 145)
        canvas.fill_text('Payoff=0', 510, 155)

def canvas_to_price(x):
    """Convert x coordinate to stock price"""
    return 0.5 + (x / CANVAS_WIDTH) * 1.0

def canvas_to_payoff(y):
    """Convert y coordinate to payoff"""
    return (CANVAS_HEIGHT/2 - y) / 100

#  event handlers
def on_mouse_down(x, y):
    global is_drawing, drawn_points
    is_drawing = True
    drawn_points = [(x, y)]

    with hold_canvas(canvas):
        canvas.fill_style = 'red'
        canvas.fill_circle(x, y, 3)  # Start point

    print(f"Started drawing at ({x:.0f}, {y:.0f})")

def on_mouse_move(x, y):
    global is_drawing, drawn_points
    if is_drawing and len(drawn_points) > 0:
        drawn_points.append((x, y))

        with hold_canvas(canvas):
            # Draw line from last point
            last_x, last_y = drawn_points[-2]
            canvas.stroke_style = 'blue'
            canvas.line_width = 3
            canvas.stroke_line(last_x, last_y, x, y)

def on_mouse_up(x, y):
    global is_drawing
    if is_drawing:
        is_drawing = False
        print(f"✓ Finished drawing! Total points: {len(drawn_points)}")

# Button handlers
def clear_drawing(btn):
    global drawn_points
    drawn_points = []
    setup_canvas()
    with out_area:
        out_area.clear_output()
    print("🧹 Canvas cleared")

def run_simulation(btn):
    global drawn_points

    with out_area:
        out_area.clear_output(wait=True)

        if len(drawn_points) < 3:
            print("⚠️ Please draw a payoff curve first! Need at least 3 points.")
            print("💡 Click and drag across the canvas to draw.")
            return

        print(f"🔄 Running Monte Carlo with {len(drawn_points)} points...")

        # Convert points to price/payoff
        prices = [canvas_to_price(x) for x, y in drawn_points]
        payoffs = [canvas_to_payoff(y) for x, y in drawn_points]

        # Sort by price for interpolation
        sorted_data = sorted(zip(prices, payoffs))
        prices_sorted = [p for p, _ in sorted_data]
        payoffs_sorted = [pf for _, pf in sorted_data]

        # Remove duplicates
        unique_prices = []
        unique_payoffs = []
        for i, (p, pf) in enumerate(zip(prices_sorted, payoffs_sorted)):
            if i == 0 or abs(p - unique_prices[-1]) > 1e-6:
                unique_prices.append(p)
                unique_payoffs.append(pf)

        if len(unique_prices) < 2:
            print("⚠️ Need more varied price points. Try drawing across more of the canvas.")
            return

        # Create interpolation
        try:
            payoff_func = interp1d(unique_prices, unique_payoffs,
                                 bounds_error=False, fill_value=0, kind='linear')
        except Exception as e:
            print(f"⚠️ Interpolation error: {e}")
            return

        # Monte Carlo simulation
        np.random.seed(42)
        Z = np.random.standard_normal(N_PATHS)
        ST = S0 * np.exp((MU - 0.5 * SIGMA**2) * T + SIGMA * np.sqrt(T) * Z)

        # Calculate payoffs
        payoffs_mc = payoff_func(ST)
        payoffs_mc = np.nan_to_num(payoffs_mc, nan=0.0)

        # Results
        mean_pf = np.mean(payoffs_mc)
        std_pf = np.std(payoffs_mc)
        var_95 = np.percentile(payoffs_mc, 5)

        # Plot results
        plt.figure(figsize=(12, 8))

        # Subplot 1: Drawn payoff function
        plt.subplot(2, 1, 1)
        price_range = np.linspace(0.5, 1.5, 200)
        payoff_range = payoff_func(price_range)
        plt.plot(price_range, payoff_range, 'b-', linewidth=2, label='Drawn Payoff')
        plt.axvline(S0, color='red', linestyle='--', alpha=0.7, label=f'S₀={S0}')
        plt.xlabel('Stock Price')
        plt.ylabel('Payoff')
        plt.title('Your Drawn Payoff Function')
        plt.grid(True, alpha=0.3)
        plt.legend()

        # Subplot 2: Monte Carlo results
        plt.subplot(2, 1, 2)
        plt.hist(payoffs_mc, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
        plt.axvline(mean_pf, color='red', linestyle='--', linewidth=2,
                   label=f'Mean: {mean_pf:.4f}')
        plt.axvline(var_95, color='orange', linestyle='--', linewidth=2,
                   label=f'95% VaR: {var_95:.4f}')
        plt.xlabel('Payoff')
        plt.ylabel('Frequency')
        plt.title(f'Monte Carlo Results ({N_PATHS:,} simulations)')
        plt.legend()
        plt.grid(True, alpha=0.3)

        # Add statistics text box
        stats_text = f'Expected Payoff: {mean_pf:.4f}\nStd Deviation: {std_pf:.4f}\n95% VaR: {var_95:.4f}'
        plt.text(0.02, 0.98, stats_text, transform=plt.gca().transAxes,
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightblue'))

        plt.tight_layout()
        plt.show()

        print("📊 Simulation Complete!")
        print(f"   Expected Payoff: {mean_pf:.4f}")
        print(f"   Volatility: {std_pf:.4f}")
        print(f"   95% VaR: {var_95:.4f}")

# Create widgets
setup_canvas()
clear_btn = Button(description='🧹 Clear', button_style='warning')
run_btn = Button(description='🎯 Run Monte Carlo', button_style='success')
out_area = Output()

# Bind events
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)
clear_btn.on_click(clear_drawing)
run_btn.on_click(run_simulation)

# Create layout
app = VBox([
    canvas,
    HBox([clear_btn, run_btn]),
    out_area
])

# Display everything
display(app)

print("🎨 READY TO DRAW!")
print("📝 Instructions:")
print("   1. Click and drag on the canvas to draw your payoff curve")
print("   2. Try drawing a call option: flat line then upward slope")
print("   3. Click 'Run Monte Carlo' to see the valuation")
print("   4. Use 'Clear' to start over")

🎨 READY TO DRAW!
📝 Instructions:
   1. Click and drag on the canvas to draw your payoff curve
   2. Try drawing a call option: flat line then upward slope
   3. Click 'Run Monte Carlo' to see the valuation
   4. Use 'Clear' to start over
Started drawing at (245, 84)
✓ Finished drawing! Total points: 62


✅ ipycanvas installed successfully


🎨 Ready to draw! Click and drag on the canvas to sketch your payoff curve.
📊 Then click 'Run Monte Carlo' to see the simulation results.
